<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/Cleaning_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np

df = pd.read_excel('/content/drive/MyDrive/awn_dataset.xlsx')
df.head(2)


,name,national_id,case_type,family_size,phone,amount_received,income_monthly,monthly_expenses,debts,housing_type,...,request_text,assistance_type,need_level,city,neighborhood,number_of_children,age,expense_to_income_ratio,recommended_org,gender
0,نجلاء عبدالباسط,5.037050e+13,ذوي إعاقة,3.0,1.183630e+09,200.0,1798.0,3050.0,0.0,سكن لدى الأقارب,...,نجلاء عبدالباسط من ذوي إعاقة مقيم في المطرية و...,disability_equipment,Low,الزقازيق,إمبابة,1.0,49.0,1.70,جمعية الأورمان,female
1,حسن كمال,6.515039e+13,أيتام,4.0,1.014614e+09,1000.0,3497.0,6328.0,8961.0,تمليك,...,حسن كمال من أيتام مقيم في إمبابة ويحتاج: تحتاج...,food,High,سوهاج,الهرم,0.0,37.0,1.81,جمعية التكافل الاجتماعي,male


In [8]:
df.shape

(60000, 21)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   name                     57035 non-null  object 
 1   national_id              57050 non-null  float64
 2   case_type                56933 non-null  object 
 3   family_size              57021 non-null  float64
 4   phone                    57124 non-null  float64
 5   amount_received          57009 non-null  float64
 6   income_monthly           57073 non-null  float64
 7   monthly_expenses         57060 non-null  float64
 8   debts                    57014 non-null  float64
 9   housing_type             56982 non-null  object 
 10  health_status            57116 non-null  object 
 11  request_text             56989 non-null  object 
 12  assistance_type          56988 non-null  object 
 13  need_level               57005 non-null  object 
 14  city                  

In [10]:
for col in df.columns:
  if df[col].dtype == "object":
    df[col] = df[col].astype(str).str.strip()
    df[col] = df[col].replace({"":np.nan, "nan": np.nan})

In [11]:
numeric_cols = [
    "family_size","amount_received","income_monthly",
    "monthly_expenses","debts","number_of_children",
    "age","expense_to_income_ratio"
]

for col in numeric_cols:
  df[col] = pd.to_numeric(df[col], errors="coerce")

In [12]:
df.loc[df["expense_to_income_ratio"] > 3, "monthly_expenses"] = \
    df["income_monthly"] * 1.8

df["expense_to_income_ratio"] = (
    df["monthly_expenses"] / df["income_monthly"]
).round(2)


text_cols = [
    "name","case_type","housing_type","health_status","request_text",
    "assistance_type","need_level","city","neighborhood",
    "recommended_org","gender"
]

df[text_cols] = df[text_cols].fillna("Unknown")


In [13]:
df = df[df["name"] != "Unknown"]

In [14]:
import re

def extract_place(text):
    match = re.search(r"مقيم في\s+([\u0600-\u06FF ]+)", str(text))
    if match:
        place = match.group(1)
        place = place.split("ويحتاج")[0].strip()
        return place
    return None

df["place"] = df["request_text"].apply(extract_place)


In [15]:
neighborhood_map = {
    "إمبابة": "الجيزة",
    "المطرية": "القاهرة",
    "مدينة نصر": "القاهرة",
    "شبرا": "القاهرة",
    "حلوان": "القاهرة",
    "العطارين": "الإسكندرية",
    "سان ستيفانو": "الإسكندرية",
    "الهرم": "الجيزة",
    "المعادي": "القاهرة",
}


In [16]:
# تحديث neighborhood
df["neighborhood"] = df["place"].apply(
    lambda x: x if x in neighborhood_map else df["neighborhood"]
)

# تحديث city باستخدام الـ mapping
df["city"] = df["place"].apply(
    lambda x: neighborhood_map[x] if x in neighborhood_map else df["city"]
)


In [17]:
df.head()

,name,national_id,case_type,family_size,phone,amount_received,income_monthly,monthly_expenses,debts,housing_type,...,assistance_type,need_level,city,neighborhood,number_of_children,age,expense_to_income_ratio,recommended_org,gender,place
0,نجلاء عبدالباسط,5.037050e+13,ذوي إعاقة,3.0,1.183630e+09,200.0,1798.0,3050.0,0.0,سكن لدى الأقارب,...,disability_equipment,Low,القاهرة,المطرية,1.0,49.0,1.70,جمعية الأورمان,female,المطرية
1,حسن كمال,6.515039e+13,أيتام,4.0,1.014614e+09,1000.0,3497.0,6328.0,8961.0,تمليك,...,food,High,الجيزة,إمبابة,0.0,37.0,1.81,جمعية التكافل الاجتماعي,male,إمبابة
2,فاطمة عبدالرحمن,3.251524e+13,أرملة,8.0,1.013125e+09,1500.0,2482.0,NaN,0.0,سكن لدى الأقارب,...,education,Low,الإسكندرية,سان ستيفانو,3.0,22.0,NaN,جمعية رسالة,female,سان ستيفانو
3,إبراهيم الهواري,4.607155e+13,أيتام,6.0,1.200902e+09,600.0,2873.0,5516.0,0.0,تمليك,...,financial,Medium,الإسكندرية,العطارين,1.0,47.0,1.92,Unknown,Unknown,العطارين
5,آمال أنور,8.965847e+13,أرملة,2.0,1.010018e+09,800.0,2530.0,5243.0,14473.0,سكن حكومي,...,food,High,القاهرة,مدينة نصر,1.0,28.0,2.07,Unknown,female,مدينة نصر


**Data** Validation Check

In [18]:
df.isnull().sum()

,0
name,0
national_id,2799
case_type,0
family_size,2819
phone,2737
amount_received,2839
income_monthly,2775
monthly_expenses,2782
debts,2842
housing_type,0


In [19]:
(df.isnull().sum() / len(df)) * 100


,0
name,0.000000
national_id,4.907513
case_type,0.000000
family_size,4.942579
phone,4.798808
amount_received,4.977645
income_monthly,4.865434
monthly_expenses,4.877707
debts,4.982905
housing_type,0.000000


In [20]:
numeric_cols = [
    "family_size", "amount_received", "income_monthly",
    "monthly_expenses", "debts", "number_of_children", "age"
]

for col in numeric_cols:
    print(col, " → Min:", df[col].min(), " | Max:", df[col].max())


family_size  → Min: 2.0  | Max: 8.0
amount_received  → Min: 0.0  | Max: 2000.0
income_monthly  → Min: 600.0  | Max: 5792.0
monthly_expenses  → Min: 726.0  | Max: 11769.0
debts  → Min: 0.0  | Max: 29998.0
number_of_children  → Min: 0.0  | Max: 8.0
age  → Min: 20.0  | Max: 65.0


In [21]:
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype(str)
    print(col, " → Count Unknown:", (df[col] == "Unknown").sum())


name  → Count Unknown: 0
case_type  → Count Unknown: 2923
housing_type  → Count Unknown: 2866
health_status  → Count Unknown: 2719
request_text  → Count Unknown: 2860
assistance_type  → Count Unknown: 2866
need_level  → Count Unknown: 2838
city  → Count Unknown: 0
neighborhood  → Count Unknown: 0
recommended_org  → Count Unknown: 2883
gender  → Count Unknown: 2874
place  → Count Unknown: 0


In [22]:
df = df[df["national_id"].notnull()]


In [23]:
df[df["expense_to_income_ratio"] > 3][
    ["name","expense_to_income_ratio","income_monthly","monthly_expenses"]
]


,name,expense_to_income_ratio,income_monthly,monthly_expenses


In [24]:
df.loc[df["expense_to_income_ratio"] > 3, "monthly_expenses"] = \
    df["income_monthly"] * 2.5


In [25]:
df["expense_to_income_ratio"] = (
    df["monthly_expenses"] / df["income_monthly"]
).round(2)


In [26]:
df[(df["age"] < 15) | (df["age"] > 90)][["name","age"]]


,name,age


In [27]:
df[df["number_of_children"] > df["family_size"]][
    ["name","family_size","number_of_children"]
]


,name,family_size,number_of_children


In [28]:
df.loc[df["number_of_children"] > df["family_size"], "family_size"] = \
    df["number_of_children"]


In [29]:
df = df[df["national_id"].notnull()]


In [30]:
df.shape

(54236, 22)

In [31]:
df["phone"].fillna("Unknown", inplace=True)


/tmp/ipython-input-2147973702.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["phone"].fillna("Unknown", inplace=True)
/tmp/ipython-input-2147973702.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["phone"].fillna("Unknown", inplace=True)


In [32]:
df.shape

(54236, 22)

In [33]:
df.drop_duplicates(subset=["national_id"], keep="first", inplace=True)


In [34]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)


In [35]:
df.shape

(54236, 22)

In [36]:
# تأكيد إن phone نص
df["phone"] = df["phone"].astype(str).str.strip()

# إزالة أي .0 ناتجة عن التحويل من float
df["phone"] = df["phone"].str.replace(r"\.0$", "", regex=True)

# الأرقام اللي طولها 10 → نضيف 0 في البداية
mask_10 = df["phone"].str.len() == 10
df.loc[mask_10, "phone"] = "0" + df.loc[mask_10, "phone"]

# أي قيمة غير 11 رقم أو فيها حروف → Unknown
mask_invalid = (~df["phone"].str.match(r"^0\d{10}$"))
df.loc[mask_invalid, "phone"] = "Unknown"


In [37]:
df.head()

,name,national_id,case_type,family_size,phone,amount_received,income_monthly,monthly_expenses,debts,housing_type,...,assistance_type,need_level,city,neighborhood,number_of_children,age,expense_to_income_ratio,recommended_org,gender,place
0,نجلاء عبدالباسط,5.037050e+13,ذوي إعاقة,3.0,Unknown,200.0,1798.0,3050.0,0.0,سكن لدى الأقارب,...,disability_equipment,Low,القاهرة,المطرية,1.0,49.0,1.70,جمعية الأورمان,female,المطرية
1,حسن كمال,6.515039e+13,أيتام,4.0,Unknown,1000.0,3497.0,6328.0,8961.0,تمليك,...,food,High,الجيزة,إمبابة,0.0,37.0,1.81,جمعية التكافل الاجتماعي,male,إمبابة
2,فاطمة عبدالرحمن,3.251524e+13,أرملة,8.0,Unknown,1500.0,2482.0,NaN,0.0,سكن لدى الأقارب,...,education,Low,الإسكندرية,سان ستيفانو,3.0,22.0,NaN,جمعية رسالة,female,سان ستيفانو
3,إبراهيم الهواري,4.607155e+13,أيتام,6.0,Unknown,600.0,2873.0,5516.0,0.0,تمليك,...,financial,Medium,الإسكندرية,العطارين,1.0,47.0,1.92,Unknown,Unknown,العطارين
5,آمال أنور,8.965847e+13,أرملة,2.0,Unknown,800.0,2530.0,5243.0,14473.0,سكن حكومي,...,food,High,القاهرة,مدينة نصر,1.0,28.0,2.07,Unknown,female,مدينة نصر


In [38]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
df.to_excel("/content/drive/MyDrive/cleaned_awn_data.xlsx", index=False)
